In [1]:
# Install requirements (run in Colab cell)
!pip install mlflow pytest scikit-learn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.2 MB/s eta 0:00:00


In [3]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 19.4 MB/s eta 0:00:00


In [6]:
# For Google Colab: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
import mlflow
import mlflow.sklearn

# --- Load processed data ---
processed_df = pd.read_csv("/content/drive/My Drive/kifiya5/processed_data.csv")

# --- Fix: Only use numeric, non-ID features for X ---
drop_cols = ['CustomerId', 'TransactionId', 'TransactionStartTime', 'Cluster']
X = processed_df.drop(drop_cols + ['is_high_risk'], axis=1, errors='ignore')
X = X.select_dtypes(include=[np.number])
y = processed_df['is_high_risk']

# --- Split data ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Define models and hyperparameters ---
models = {
    "logistic_regression": (
        LogisticRegression(max_iter=1000, random_state=42),
        {"C": [0.1, 1, 10]}
    ),
    "random_forest": (
        RandomForestClassifier(random_state=42),
        {"n_estimators": [50, 100], "max_depth": [5, 10]}
    )
}

best_score = 0
best_model = None
best_model_name = ""
best_params = None

mlflow.set_experiment("credit-risk-model")

for model_name, (model, params) in models.items():
    with mlflow.start_run(run_name=model_name):
        clf = GridSearchCV(model, params, scoring='f1', cv=3, n_jobs=-1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_proba = clf.predict_proba(X_test)[:, 1] if hasattr(clf, "predict_proba") else None

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan

        # Log metrics and params
        mlflow.log_params(clf.best_params_)
        mlflow.log_metrics({
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1_score": f1,
            "roc_auc": roc_auc
        })
        mlflow.sklearn.log_model(clf.best_estimator_, model_name)

        # Track best model
        if f1 > best_score:
            best_score = f1
            best_model = clf.best_estimator_
            best_model_name = model_name
            best_params = clf.best_params_

# --- Register the best model ---
with mlflow.start_run(run_name="register_best_model"):
    mlflow.sklearn.log_model(best_model, "best_model")
    mlflow.register_model(
        "runs:/{}/best_model".format(mlflow.active_run().info.run_id),
        "CreditRiskBestModel"
    )

print(f"Best model: {best_model_name} with params: {best_params} and F1: {best_score:.4f}")

# 3. Example Unit Tests
"""
import pytest
import pandas as pd
from your_module import customer_aggregator  # adjust import as needed

def test_customer_aggregator_sum():
    df = pd.DataFrame({
        'CustomerId': [1, 1, 2],
        'Amount': [10, 20, 30],
        'TransactionId': [100, 101, 102]
    })
    customer_aggregator(df, fit=True)
    result = customer_aggregator(df)
    assert 'Total_Transaction_Amount' in result.columns
    assert result[result['CustomerId'] == 1]['Total_Transaction_Amount'].iloc[0] == 30

def test_customer_aggregator_unique_transactions():
    df = pd.DataFrame({
        'CustomerId': [1, 1, 2],
        'Amount': [10, 20, 30],
        'TransactionId': [100, 101, 102]
    })
    customer_aggregator(df, fit=True)
    result = customer_aggregator(df)
    assert 'Unique_Transaction_Count' in result.columns
    assert result[result['CustomerId'] == 1]['Unique_Transaction_Count'].iloc[0] == 2
"""

2025/07/01 19:15:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 19:15:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/01 19:17:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 19:17:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/01 19:17:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 19:17:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'CreditRiskBestModel'.
2025/07/01 19:17:34 WARNING mlflow.tra

Best model: logistic_regression with params: {'C': 0.1} and F1: 1.0000


Created version '1' of model 'CreditRiskBestModel'.


"\nimport pytest\nimport pandas as pd\nfrom your_module import customer_aggregator  # adjust import as needed\n\ndef test_customer_aggregator_sum():\n    df = pd.DataFrame({\n        'CustomerId': [1, 1, 2],\n        'Amount': [10, 20, 30],\n        'TransactionId': [100, 101, 102]\n    })\n    customer_aggregator(df, fit=True)\n    result = customer_aggregator(df)\n    assert 'Total_Transaction_Amount' in result.columns\n    assert result[result['CustomerId'] == 1]['Total_Transaction_Amount'].iloc[0] == 30\n\ndef test_customer_aggregator_unique_transactions():\n    df = pd.DataFrame({\n        'CustomerId': [1, 1, 2],\n        'Amount': [10, 20, 30],\n        'TransactionId': [100, 101, 102]\n    })\n    customer_aggregator(df, fit=True)\n    result = customer_aggregator(df)\n    assert 'Unique_Transaction_Count' in result.columns\n    assert result[result['CustomerId'] == 1]['Unique_Transaction_Count'].iloc[0] == 2\n"